# Supplementary Information and Figures

This file contains information about the data sources and pre-processing steps in
Richardson & Eddy, 2023. All figures can be reproduced using this code.

In [ ]:
# Import packages for data wrangling
import numpy as np
import pandas as pd
import itertools as it
import functools as ft

# Import custom functions
import import_data as orfdata
import parameters_set as orfparams
import output_plots as orfplt

# Download data

1. Download reference genome sequences (fasta) and ncRNA sequences
2. Use the provided annotation files (gtf)

### SARS-CoV-2
1. Download genome version ASM985889v3: https://covid-19.ensembl.org/info/data/index.html
2. Use `data/coronavirus/annotations.gtf`

### E. coli

1. Download genome version ASM584v2: https://bacteria.ensembl.org/Escherichia_coli_str_k_12_substr_mg1655_gca_000005845/Info/Index/
2. Use `data/ecoli/annotations.gtf`
    - NOTE: I manually updated the +1 PRF annotation that skipped the gap nucleotide to include it (merged exons)
    - NOTE: I added the 5’UTRs and 3’UTRs from RegulonDB to the annotations gtf file

### S. cerevisiae

1. Download genome version R64-1-1: https://ensembl.org/Saccharomyces_cerevisiae/Info/Index
2. Use `data/scerevisiae/annotations.gtf`
    - NOTE: I manually updated the +1 PRF annotations that skipped the gap nucleotide to include it (merged exons)
and added the first exon (single codon) for RPL20A and RPL20B
    - NOTE: I added the 5’UTRs and 3’UTRs from Nagalakshmi et al. to the annotations gtf file

### D. melanogaster

1. Download genome version BDGP6.32: https://ensembl.org/Drosophila_melanogaster/Info/Index
2. Use `data/dmelanogaster/annotations.gtf`

### D. rerio

1. Download genome version GRCz11: https://ensembl.org/Danio_rerio/Info/Index
2. Use `data/drerio/annotations.gtf`

# Process data

1. Align riboseq reads to the genome. Trim adapter sequences and filter by read quality and length as appropriate. Remove hits to ncRNA and contaminating genomes as appropriate. This step is not performed by ORFeus.

2. Generate bedgraph (.bg) or wiggle (.wig) format files with the number of reads at each P-site. Existing software tools can help offset reads to the P-site and export strand-specific bg/wig files. Our offsetting was done using Shoelaces (https://bitbucket.org/valenlab/shoelaces/). This step is not performed by ORFeus.

3. Run the first step of ORFeus to process the data and build the model.

### SARS-CoV-2

1. Align data from Finkel et al, 2021: 
`sbatch supplements/alignment_star.sh -a CTGTAGGCACCATCAAT -q 20 -m 28 -M 30 -r SRR12216748 -r SRR12216749 -r SRR12216750 data/coronavirus/ncrna.fa data/hsapiens/dna.fa data/coronavirus/dna.fa data/coronavirus/annotations.gtf data/coronavirus/`

2. Offset reads to P-site: shift reads 5'+14nt

3. Process data:
`python orfeus/build.py data/coronavirus/Finkel2021_forward.wig data/coronavirus/Finkel2021_reverse.wig data/coronavirus/dna.fa data/coronavirus/annotations.gtf -o data/coronavirus/`

### E. coli

1. Align data from Hwang & Buskirk, 2017:
`sbatch supplements/alignment_bowtie.sh -u -a CTGTAGGCACCATCAATAGATCGGAAGAGCACACGTCTGAACTCCAGTCA -q 20 -m 20 -M 40 -r SRR4023281 data/ecoli/ladder.fa data/ecoli/ncrna.fa data/ecoli/dna.fa data/ecoli/annotations.gtf data/ecoli/`

2. Offset reads to P-site: shift reads 3'-3nt

3. Process data:
`python orfeus/build.py data/ecoli/Hwang2017_forward.wig data/ecoli/Hwang2017_reverse.wig data/ecoli/dna.fa data/ecoli/annotations.gtf -o data/ecoli/`

### S. cerevisiae

1. Align data from Wu et al, 2019:
`sbatch supplements/alignment_bowtie.sh -u -a CTGTAGGCACCATCAAT -q 20 -m 28 -M 28 -r SRR7241903 -r SRR7241904 data/scerevisiae/ladder.fa data/scerevisiae/ncrna.fa data/scerevisiae/dna.fa data/scerevisiae/annotations.gtf data/scerevisiae/`

2. Offset reads to P-site: shift reads 5'+14nt

3. Process data:
`python orfeus/build.py data/scerevisiae/Wu2019_forward.wig data/scerevisiae/Wu2019_reverse.wig data/scerevisiae/dna.fa data/scerevisiae/annotations.gtf -o data/scerevisiae/`

### D. melanogaster

1. Align data from Dunn et al, 2013:
`sbatch supplements/alignment_star.sh -a CTGTAGGCACCATCAAT -q 20 -m 20 -M 40 -r SRR942868 -r SRR942869 -r SRR942870 -r SRR942871 -r SRR942874 -r SRR942875 -r SRR942876 -r SRR942877 data/dmelanogaster/ladder.fa data/dmelanogaster/ncrna.fa data/dmelanogaster/dna.fa data/dmelanogaster/annotations.gtf data/dmelanogaster/`

2. Offset reads to P-site: shift reads 5'+16nt

3. Process data:
`python orfeus/build.py data/dmelanogaster/Dunn2013_forward.wig data/dmelanogaster/Dunn2013_reverse.wig data/dmelanogaster/dna.fa data/dmelanogaster/annotations.gtf -o data/dmelanogaster/`

### D. rerio

1. Align data from Dunn et al, 2013:
`sbatch alignment_star.sh -a AGATCGGAAGAGCACACGTCT -q 20 -m 20 -M 30 -r SRR1062294 -r SRR1062295 -r SRR1062296 -r SRR1062297 -r SRR1062298 -r SRR1062299 -r SRR1062300 -r SRR1062301 -r SRR1062302 ../data/drerio/ladder.fa ../data/drerio/ncrna.fa ../data/drerio/dna.fa ../data/drerio/annotations.gtf ../data/drerio/`

2. Offset reads to P-site: shift reads 5'+12nt

3. Process data:
`python orfeus/build.py data/drerio/Bazzini2014_forward.wig data/drerio/Bazzini2014_reverse.wig data/drerio/dna.fa data/drerio/annotations.gtf -o data/drerio/`

# Import data

Import the processed data and set the parameters.

In [ ]:
# Set data file (choose one processed data set)
data_file = '../data/coronavirus/data.txt.gz'
#data_file = '../data/ecoli/data.txt.gz'
#data_file = '../data/scerevisiae/data.txt.gz'
#data_file = '../data/dmelanogaster/data.txt.gz'
#data_file = '../data/drerio/data.txt.gz'

# Import the data set
data_df = orfdata.read_data_file(data_file)

# Preview the data set
data_df[data_df['feature']=='ORF']

# Set parameters

This step is the same for all data sets. This section and all following plotting code must be run for one dataset at a time.

In [ ]:
# Constants
NUCLEOTIDES = ['A','C','G','T']
CODONS = [''.join(i) for i in it.product(NUCLEOTIDES, repeat = 3)]

In [ ]:
# Probability of PRF
alpha = 1e-5          # Suggested 1e-5

# Probability of -1 PRF given a PRF
# 0.5 indicates +1 and -1 frameshifts are equally likely 
beta = 0.5            # Suggested 0.5

# Probability of stop codon readthrough
gamma = 1e-4          # Suggested 1e-4

# Probability of uORFs or dORFs
delta = 1e-3          # Suggested 1e-3

# Probabiliy of multiple non-overlapping ORFs
zeta = 1e-10          # Suggested 1e-10

In [ ]:
# Fraction of transcripts expected to have 5'UTRs
f_5UTR = None         # Default is to calculate from data set

# Fraction of transcripts expected to have 3'UTRs
f_3UTR = None         # Default is to calculate from data set

# Expected mean length of 5'UTRs (nts)
len_5UTR = None       # Default is to calculate from data set

# Expected mean length of 3'UTRs (nts)
len_3UTR = None       # Default is to calculate from data set

# Expected mean length of ORFs (nts)
len_orf = None        # Default is to calculate from data set

# Expected mean length of uORFs (nts)
len_uorf = 50         # Suggested 50

# Expected mean length of dORFs (nts)
len_dorf = 50         # Suggested 50

# Start codons to consider
#start_codons = None  # Calculate from data set
start_codons=['ATG']

# Stop codons to consider
#stop_codons = None  # Calculate from data set
stop_codons=['TAA','TAG','TGA']

# Sense codons to consider (be sure to exclude stop codons)
#sense_codons = None  # Calculate from data set
sense_codons=list(filter(lambda i: i not in ['TAA','TAG','TGA'], CODONS))

In [ ]:
# Number of bins to use for rho emissions
num_bins = 25        # Suggested 25

# Whether to bin rho emissions in logspace
logspace = False     # Suggested False

# Set parameters based on only transcripts with at 
# least this many mean reads per transcript (minimum)
min_coverage = -1       # Suggested -1

# Set parameters based on only transcripts with at 
# most this many mean reads per transcript (maximum)
max_coverage = np.inf   # Suggested np.inf

# Whether to pool all codons of a given type (start, stop, sense)
# when calculating rho emissions
pool = False         # Suggested False

# Whether to fit a log-normal distribution to the rho emissions
fit = False          # Suggested False

In [ ]:
# Set model parameters
parameters_h1 = orfparams.set_parameters(data_df,
                                         f_5UTR,
                                         f_3UTR,
                                         len_5UTR,
                                         len_3UTR,
                                         len_orf,
                                         len_uorf,
                                         len_dorf,
                                         start_codons,
                                         sense_codons,
                                         stop_codons,
                                         num_bins,
                                         logspace,
                                         min_coverage,
                                         max_coverage,
                                         pool,
                                         fit,
                                         [alpha, beta, gamma, delta, zeta])

parameters_h0 = orfparams.set_parameters(data_df,
                                         f_5UTR,
                                         f_3UTR,
                                         len_5UTR,
                                         len_3UTR,
                                         len_orf,
                                         len_uorf,
                                         len_dorf,
                                         start_codons,
                                         sense_codons,
                                         stop_codons,
                                         num_bins,
                                         logspace,
                                         min_coverage,
                                         max_coverage,
                                         pool,
                                         fit,
                                         [0, 0, 0, 0, 0])

# Plot results

### SARS-CoV-2

Run the above code for the coronavirus data set before running this section.

In [ ]:
# Total read count for whole data set
print('Total read count: %i' % sum(data_df['reads']))

# Metagene plot
window = 30
signal_start, signal_stop = orfplt.calculate_metagene_signal(data_df, window)
orfplt.plot_metagene_signal(signal_start, signal_stop, window, '../data/coronavirus/Finkel2021_metagene')

In [ ]:
# ORF1ab (-1 PRF)
transcript_id = 'ENSSAST00005000002'
transcript_name = 'ORF1ab'

orfplt.orfeus(data_df, transcript_id, transcript_name, parameters_h0, parameters_h1, 
              suppress=False, debug=False, subcodon=True)

### E. coli

Run the above code for the E. coli data set before running this section.

In [ ]:
# Total read count for whole data set
print('Total read count: %i' % sum(data_df['reads']))

# Metagene plot
window = 30
signal_start, signal_stop = orfplt.calculate_metagene_signal(data_df, window)
orfplt.plot_metagene_signal(signal_start, signal_stop, window, '../data/ecoli/Hwang2017_metagene')


In [ ]:
# prfB (+1 PRF)
transcript_id = 'AAC75929'
transcript_name = 'prfB'

orfplt.orfeus(data_df, transcript_id, transcript_name, parameters_h0, parameters_h1, 
              suppress=False, debug=False, subcodon=True)

### S. cerevisiae

Run the above code for the S. cerevisiae data set before running this section.

In [ ]:
# Total read count for whole data set
print('Total read count: %i' % sum(data_df['reads']))

# Metagene plot
window = 30
signal_start, signal_stop = orfplt.calculate_metagene_signal(data_df, window)
orfplt.plot_metagene_signal(signal_start, signal_stop, window, '../data/scerevisiae/Wu2019_metagene')

In [ ]:
# Total read count for single housekeeping gene
df = data_df[data_df['transcript_name']=='PDA1']
print('Total read count: %i' % sum(df['reads']))

# Single-gene plot
window = 30
signal_start, signal_stop = orfplt.calculate_metagene_signal(df, window)
orfplt.plot_metagene_signal(signal_start, signal_stop, window, '../data/scerevisiae/Wu2019_PDA1')

In [ ]:
# GCN4 (uORFs)
transcript_id = 'YEL009C_mRNA'
transcript_name = 'GCN4'

orfplt.orfeus(data_df, transcript_id, transcript_name, parameters_h0, parameters_h1, 
              suppress=False, debug=False, subcodon=True)

### D. melanogaster

Run the above code for the D. melanogaster data set before running this section.

In [ ]:
# Total read count for whole data set
print('Total read count: %i' % sum(data_df['reads']))

# Metagene plot
window = 30
signal_start, signal_stop = orfplt.calculate_metagene_signal(data_df, window)
orfplt.plot_metagene_signal(signal_start, signal_stop, window, '../data/dmelanogaster/Dunn2013_metagene')

In [ ]:
# hdc-RC (SCR)
transcript_id = 'FBtr0085621'
transcript_name = 'hdc-RC'

orfplt.orfeus(data_df, transcript_id, transcript_name, parameters_h0, parameters_h1, 
              suppress=False, debug=False, subcodon=True)

### D. rerio

Run the above code for the D. rerio data set before running this section.

In [ ]:
# Total read count for whole data set
print('Total read count: %i' % sum(data_df['reads']))

# Metagene plot
window = 30
signal_start, signal_stop = orfplt.calculate_metagene_signal(data_df, window)
orfplt.plot_metagene_signal(signal_start, signal_stop, window, '../data/drerio/Bazzini2014_metagene')

In [ ]:
# rrm1 (dORF)
transcript_id = 'ENSDART00000012091'
transcript_name = 'rrm1'

orfplt.orfeus(data_df, transcript_id, transcript_name, parameters_h0, parameters_h1, 
              suppress=False, debug=False, subcodon=True)

# Plot simulation

Results from the simulation using parameters trained on S. cerevisiae (Wu et al, 2019) data are in `data/scerevisiae/coverage`.

These results were generated with `python orfeus/build.py data/scerevisiae/Wu2019_forward.wig data/scerevisiae/Wu2019_reverse.wig data/scerevisiae/dna.fa data/scerevisiae/annotations.gtf -o data/scerevisiae/ -c --iters 100`

In [ ]:
pPRF = pd.read_table('../data/scerevisiae/coverage/sensitivity_pPRF.txt', sep=' ', index_col=0, usecols=[0,3])
mPRF = pd.read_table('../data/scerevisiae/coverage/sensitivity_mPRF.txt', sep=' ', index_col=0, usecols=[0,3])
SCR  = pd.read_table('../data/scerevisiae/coverage/sensitivity_SCR.txt', sep=' ', index_col=0, usecols=[0,3])
uORFdORF = pd.read_table('../data/scerevisiae/coverage/sensitivity_uORFdORF.txt', sep=' ', index_col=0, usecols=[0,3])
ORF = pd.read_table('../data/scerevisiae/coverage/sensitivity_ORF.txt', sep=' ', index_col=0, usecols=[0,3])

results = ft.reduce(lambda A,B: pd.merge(A, B, left_index=True, right_index=True), 
                    [uORFdORF, pPRF, mPRF, SCR, ORF])

sensitivities = results.to_dict()
orfplt.plot_results(sensitivities, 'Sensitivity', '../data/scerevisiae/coverage/sensitivity')


In [ ]:
pPRF = pd.read_table('../data/scerevisiae/coverage/specificity_pPRF.txt', sep=' ', index_col=0, usecols=[0,3])
mPRF = pd.read_table('../data/scerevisiae/coverage/specificity_mPRF.txt', sep=' ', index_col=0, usecols=[0,3])
SCR  = pd.read_table('../data/scerevisiae/coverage/specificity_SCR.txt', sep=' ', index_col=0, usecols=[0,3])
uORFdORF = pd.read_table('../data/scerevisiae/coverage/specificity_uORFdORF.txt', sep=' ', index_col=0, usecols=[0,3])

results = ft.reduce(lambda A,B: pd.merge(A, B, left_index=True, right_index=True), 
                    [uORFdORF, pPRF, mPRF, SCR])

specificities = results.to_dict()
orfplt.plot_results(specificities, 'Specificity', '../data/scerevisiae/coverage/specificity')


In [ ]:
results = pd.read_table('../data/scerevisiae/coverage/coverage_counts.txt', sep=' ', index_col=0, usecols=[0,4])

coverage_counts = results['least'].to_dict()
orfplt.plot_transcript_coverage(coverage_counts, '../data/scerevisiae/coverage/coverage_counts')